
**ライブラリーの輸入**

**データのクリーニングと操作:** pandas, numpy, datetime, sys, os, platform, 
subprocess
**可視化:** plotly, matplotlib, seaborn

*その後、ファイルをインポートし、データを調整する*

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from datetime import datetime

import sys
from matplotlib import dates
import seaborn as sns
import os
import platform
import subprocess

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

In [ ]:
# ファイル輸入
from google.colab import files

uploaded = files.upload()
uploaded2 = files.upload()

Saving MC24_ver2.txt to MC24_ver2.txt


Saving Status.xlsx to Status.xlsx


In [ ]:
# データ１Headerの変化

Data = pd.read_csv('MC24_ver2.txt',names=['Model','Time','Light_1','Light_2','Light_3'],)

# データ２Headerの変化

Status = pd.read_excel('Status.xlsx',names=['Light_1','Light_2','Light_3','Status'],)

In [ ]:
# データ確認
Data

,Model,Time,Light_1,Light_2,Light_3
0,MC24,1,0,1,0
1,MC24,92,0,0,0
2,MC24,988,0,0,1
3,MC24,998,0,0,0
4,MC24,1915,0,0,1
...,...,...,...,...,...
90,MC24,52959,0,1,0
91,MC24,54081,0,0,0
92,MC24,54082,0,1,1
93,MC24,54082,0,1,0


In [ ]:
# Datatype 確認

display(Data.dtypes)
display(Status.dtypes)

Model      object
Time        int64
Light_1     int64
Light_2     int64
Light_3     int64
dtype: object

Light_1     int64
Light_2     int64
Light_3     int64
Status     object
dtype: object

必要な開始時刻に時刻を設定

1.   Timing2 という新しい時刻データの作成
2.   Datatype を integer に変更し、簡単に追加できるようにする。
その後、Timestampに変更する。



In [ ]:
 timestr = '08:31:59'

ftr = [3600,60,1]

t = sum([a*b for a,b in zip(ftr, map(int,timestr.split(':')))])
t

# Time starting from 8:32
from datetime import datetime

Data['Timing2'] = t+1
Data.Timing2[0] = t
Data

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


,Model,Time,Light_1,Light_2,Light_3,Timing2
0,MC24,1,0,1,0,30719
1,MC24,92,0,0,0,30720
2,MC24,988,0,0,1,30720
3,MC24,998,0,0,0,30720
4,MC24,1915,0,0,1,30720
...,...,...,...,...,...,...
90,MC24,52959,0,1,0,30720
91,MC24,54081,0,0,0,30720
92,MC24,54082,0,1,1,30720
93,MC24,54082,0,1,0,30720


In [ ]:
Data['Timing2'] = Data['Timing2'] + Data['Time']
Data

,Model,Time,Light_1,Light_2,Light_3,Timing2
0,MC24,1,0,1,0,30720
1,MC24,92,0,0,0,30812
2,MC24,988,0,0,1,31708
3,MC24,998,0,0,0,31718
4,MC24,1915,0,0,1,32635
...,...,...,...,...,...,...
90,MC24,52959,0,1,0,83679
91,MC24,54081,0,0,0,84801
92,MC24,54082,0,1,1,84802
93,MC24,54082,0,1,0,84802


In [ ]:
# 時刻を hh:mm:ssフォーマットに変更

m,s = Data['Timing2'].divmod(60)
h,m = m.divmod(60)
Data['Timing2'] = (h.astype(str).str.zfill(2)+':'+m.astype(str).str.zfill(2)+':'+s.astype(str).str.zfill(2))
Data
data = Data.drop(['Time'],axis=1)

In [ ]:
# データの結合
merge =  pd.merge(data, Status, on = ["Light_1", "Light_2", "Light_3"], how = "left")
merge['Timing2'] = pd.to_datetime(merge['Timing2'])
merge

,Model,Light_1,Light_2,Light_3,Timing2,Status
0,MC24,0,1,0,2022-11-14 08:32:00,加工終了
1,MC24,0,0,0,2022-11-14 08:33:32,電源は入っているが非稼働／非稼働
2,MC24,0,0,1,2022-11-14 08:48:28,加工中
3,MC24,0,0,0,2022-11-14 08:48:38,電源は入っているが非稼働／非稼働
4,MC24,0,0,1,2022-11-14 09:03:55,加工中
...,...,...,...,...,...,...
90,MC24,0,1,0,2022-11-14 23:14:39,加工終了
91,MC24,0,0,0,2022-11-14 23:33:21,電源は入っているが非稼働／非稼働
92,MC24,0,1,1,2022-11-14 23:33:22,ー
93,MC24,0,1,0,2022-11-14 23:33:22,加工終了


In [ ]:
# Light 2 Start and end time
Light_2 = merge[["Model","Light_2" ,"Timing2"]]
Light_2

t = Light_2.loc[0]
col_names = ["Model", "Light_2", "Start_time", "End_time"]

df = pd.DataFrame(columns=col_names)

for index, row in Light_2.iterrows():
  if row["Light_2"] != t["Light_2"]:
    p = [[t["Model"], t["Light_2"], t["Timing2"], row["Timing2"]]]
    t_a = pd.DataFrame(p, columns=col_names)
    df = pd.concat([df, t_a], axis=0)
    t = row

p = [[t["Model"], t["Light_2"], t["Timing2"], Light_2.iloc[-1]["Timing2"]]]
t_a = pd.DataFrame(p, columns=col_names)
df = pd.concat([df, t_a], axis=0)

df2 = df[df["Light_2"] != 0]

df2

,Model,Light_2,Start_time,End_time
0,MC24,1,2022-11-14 08:32:00,2022-11-14 08:33:32
0,MC24,1,2022-11-14 11:37:13,2022-11-14 11:47:20
0,MC24,1,2022-11-14 13:46:57,2022-11-14 14:27:33
0,MC24,1,2022-11-14 15:32:53,2022-11-14 16:50:13
0,MC24,1,2022-11-14 17:55:32,2022-11-14 18:08:47
0,MC24,1,2022-11-14 19:14:04,2022-11-14 19:27:19
0,MC24,1,2022-11-14 20:32:34,2022-11-14 20:47:48
0,MC24,1,2022-11-14 21:53:05,2022-11-14 22:09:23
0,MC24,1,2022-11-14 23:14:38,2022-11-14 23:33:21
0,MC24,1,2022-11-14 23:33:22,2022-11-14 23:34:02


In [ ]:
# Light 3 Start and end time
Light_3 = merge[["Model","Light_3" ,"Timing2"]]
Light_3

t = Light_3.loc[0]
col_names = ["Model", "Light_3", "Start_time", "End_time"]

df = pd.DataFrame(columns=col_names)

for index, row in Light_3.iterrows():
  if row["Light_3"] != t["Light_3"]:
     p = [[t["Model"], t["Light_3"], t["Timing2"], row["Timing2"]]]
     t_a = pd.DataFrame(p, columns=col_names)
     df = pd.concat([df, t_a], axis=0)
     t = row

p = [[t["Model"], t["Light_3"], t["Timing2"], Light_3.iloc[-1]["Timing2"]]]
t_a = pd.DataFrame(p, columns=col_names)
df = pd.concat([df, t_a], axis=0)

df3 = df[df["Light_3"] != 0]

df3

,Model,Light_3,Start_time,End_time
0,MC24,1,2022-11-14 08:48:28,2022-11-14 08:48:38
0,MC24,1,2022-11-14 09:03:55,2022-11-14 09:03:56
0,MC24,1,2022-11-14 09:37:01,2022-11-14 09:37:10
0,MC24,1,2022-11-14 10:14:04,2022-11-14 10:14:04
0,MC24,1,2022-11-14 10:22:10,2022-11-14 10:22:10
0,MC24,1,2022-11-14 10:28:17,2022-11-14 10:28:21
0,MC24,1,2022-11-14 10:29:28,2022-11-14 10:29:38
0,MC24,1,2022-11-14 10:29:38,2022-11-14 10:29:39
0,MC24,1,2022-11-14 10:29:40,2022-11-14 10:29:43
0,MC24,1,2022-11-14 10:29:45,2022-11-14 10:29:45


In [ ]:
# Light 1 Start and end time
Light_1 = merge[["Model","Light_1" ,"Timing2"]]
Light_1

t = Light_1.loc[0]
col_names = ["Model", "Light_1", "Start_time", "End_time"]

df = pd.DataFrame(columns=col_names)

for index, row in Light_1.iterrows():
    if row["Light_1"] != t["Light_1"]:
        p = [[t["Model"], t["Light_1"], t["Timing2"], row["Timing2"]]]
        t_a = pd.DataFrame(p, columns=col_names)
        df = pd.concat([df, t_a], axis=0)
        t = row

p = [[t["Model"], t["Light_1"], t["Timing2"], Light_1.iloc[-1]["Timing2"]]]
t_a = pd.DataFrame(p, columns=col_names)
df = pd.concat([df, t_a], axis=0)

df1 = df[df["Light_1"] != 0]

df1

,Model,Light_1,Start_time,End_time
0,MC24,1,2022-11-14 09:05:06,2022-11-14 09:05:07
0,MC24,1,2022-11-14 09:12:52,2022-11-14 09:12:55
0,MC24,1,2022-11-14 11:19:22,2022-11-14 11:28:39
0,MC24,1,2022-11-14 21:28:09,2022-11-14 21:48:39


In [ ]:
# Status Time 
ST = merge[["Model","Status" ,"Timing2"]]
ST

t = ST.loc[0]
col_names = ["Model", "Status", "Start_time", "End_time"]

df = pd.DataFrame(columns=col_names)

for index, row in ST.iterrows():
    if row["Status"] != t["Status"]:
        p = [[t["Model"], t["Status"], t["Timing2"], row["Timing2"]]]
        t_a = pd.DataFrame(p, columns=col_names)
        df = pd.concat([df, t_a], axis=0)
        t = row

p = [[t["Model"], t["Status"], t["Timing2"], ST.iloc[-1]["Timing2"]]]
t_a = pd.DataFrame(p, columns=col_names)
df = pd.concat([df, t_a], axis=0)

df_status = df[df["Status"] != 0]

df_status


,Model,Status,Start_time,End_time
0,MC24,加工終了,2022-11-14 08:32:00,2022-11-14 08:33:32
0,MC24,電源は入っているが非稼働／非稼働,2022-11-14 08:33:32,2022-11-14 08:48:28
0,MC24,加工中,2022-11-14 08:48:28,2022-11-14 08:48:38
0,MC24,電源は入っているが非稼働／非稼働,2022-11-14 08:48:38,2022-11-14 09:03:55
0,MC24,加工中,2022-11-14 09:03:55,2022-11-14 09:03:56
...,...,...,...,...
0,MC24,加工終了,2022-11-14 23:14:39,2022-11-14 23:33:21
0,MC24,電源は入っているが非稼働／非稼働,2022-11-14 23:33:21,2022-11-14 23:33:22
0,MC24,ー,2022-11-14 23:33:22,2022-11-14 23:33:22
0,MC24,加工終了,2022-11-14 23:33:22,2022-11-14 23:34:02


**インタラクティブガントチャートの作成**

plotlyを使ってサブプロットを作成し、1本のX軸にグラフを追記

In [ ]:
# 可視化コード
fig_sub = make_subplots(
    rows=4, shared_xaxes=True,
    subplot_titles=("light_1", "light_2", "light_3", "status"),
    
)
fig_1 = px.timeline(
    df1,
    x_start = "Start_time",
    x_end = "End_time",
    y = "Model",
    color = "Light_1"
)
fig_2 = px.timeline(
    df2,
    x_start = "Start_time",
    x_end = "End_time",
    y = "Model",
    color = "Light_2"
)
fig_3 = px.timeline(
    df3,
    x_start = "Start_time",
    x_end = "End_time",
    y = "Model",
    color = "Light_3"
)
fig_4 = px.timeline(
    df_status,
    x_start = "Start_time",
    x_end = "End_time",
    y = "Model",
    color = "Status"
)
fig_sub.append_trace(fig_1["data"][0],row=1,col=1)
fig_sub.append_trace(fig_2["data"][0], row=2, col=1)
fig_sub.append_trace(fig_3["data"][0], row=3, col=1)
fig_sub.append_trace(fig_4["data"][0], row=4, col=1)
fig_sub.append_trace(fig_4["data"][1], row=4, col=1)
fig_sub.append_trace(fig_4["data"][2], row=4, col=1)
fig_sub.append_trace(fig_4["data"][3], row=4, col=1)
fig_sub.append_trace(fig_4["data"][4], row=4, col=1)
fig_sub.append_trace(fig_4["data"][5], row=4, col=1)

# グラフ全体とホバーのフォントサイズ変更
# fig.update_layout(font_size=20, hoverlabel_font_size=15)
# グラフのレイアウト変更
fig_sub.update_layout(
    # 凡例の位置変更
    legend=dict(
        xanchor='left',
        yanchor='bottom',
        x=0,
        y=-0.2,
        orientation='h',
    ),
    # グラフのサイズ変更
    width=1300,
    height=1000,
    # 縦軸ラベル削除
    yaxis = dict(
        title=None,
    )
)
fig_sub.update_xaxes(
    # グラフ上にレンジセレクターを追加
    rangeselector=dict(
        buttons=list([
            dict(
                count=1,
                label='1m',
                step='month',
                stepmode='backward'
            ),
            dict(
                count=7,  # 数値
                label='1w',  # ボタンラベル
                step='day',  # 単位
                stepmode='backward'  # 現時点より以前
            ),
            dict(
                count=1,
                label='1d',
                step='day',
                stepmode='backward'
            ),
            dict(step='all')
        ])
    ),
)

fig_sub.update_xaxes(type="date")
fig_sub.show()

**センサーステータスの経過時間を算出**

In [ ]:
d1 = pd.to_datetime(df_status["Start_time"])
d2 = pd.to_datetime(df_status['End_time'])
df_status['Elapsedtime'] = d2-d1
df_status

,Model,Status,Start_time,End_time,Elapsedtime
0,MC24,加工終了,2022-11-14 08:32:00,2022-11-14 08:33:32,0 days 00:01:32
0,MC24,電源は入っているが非稼働／非稼働,2022-11-14 08:33:32,2022-11-14 08:48:28,0 days 00:14:56
0,MC24,加工中,2022-11-14 08:48:28,2022-11-14 08:48:38,0 days 00:00:10
0,MC24,電源は入っているが非稼働／非稼働,2022-11-14 08:48:38,2022-11-14 09:03:55,0 days 00:15:17
0,MC24,加工中,2022-11-14 09:03:55,2022-11-14 09:03:56,0 days 00:00:01
...,...,...,...,...,...
0,MC24,加工終了,2022-11-14 23:14:39,2022-11-14 23:33:21,0 days 00:18:42
0,MC24,電源は入っているが非稼働／非稼働,2022-11-14 23:33:21,2022-11-14 23:33:22,0 days 00:00:01
0,MC24,ー,2022-11-14 23:33:22,2022-11-14 23:33:22,0 days 00:00:00
0,MC24,加工終了,2022-11-14 23:33:22,2022-11-14 23:34:02,0 days 00:00:40


In [ ]:
# Elapsed time for status data
status_data2 = df_status[["Model","Status", "Elapsedtime"]].groupby(["Model","Status"]).sum()
status_data2 = status_data2.reset_index()
# status_data2
# # status_data["elapsed_time"].max()
status3 = status_data2[['Status','Elapsedtime']]
status3

,Status,Elapsedtime
0,アラーム発生、加工中,0 days 00:29:47
1,アラーム発生／非常停止ボタンが押された時,0 days 00:00:04
2,ー,0 days 00:00:01
3,加工中,0 days 08:13:35
4,加工終了,0 days 03:26:59
5,電源は入っているが非稼働／非稼働,0 days 02:51:36


In [ ]:
# fig2 = px.pie(df, values=status3['Elapsedtime'], names=status3['Status'], title='円グラフによる稼働率監視')
# fig2 = px.pie(df,values=status3['Elapsedtime'], names=status3['Status'], title = '円グラフによるステータス経過時間')
# # config = dict({'scrollZoom': False})
# config = {'responsive': False}
fig2 = go.Figure()
fig2.add_trace(
    go.Pie(labels = status3['Status'],
           values = status3['Elapsedtime'])
)

fig2.update_layout(legend=dict(
    yanchor="bottom",
    y=-1,
    xanchor="left",
    x=0.01
))
button_layer_1_height = 1.12
button_layer_2_height = 0.92
button_layer_3_height = 0.72
button_layer_4_height = 0.52
button_layer_5_height = 0.32
button_layer_6_height = 0.12

fig2.update_layout(
    updatemenus=[
        dict(
            buttons=list([
                dict(
                    args=["Status", "1"],
                    label="アラーム発生、加工中 	00:29:47",
                    method="restyle"
                )
            ]),
            type = "buttons",
            direction="right",
            pad={"r": 50, "t": 50},
            showactive=True,
            x=0.9,
            xanchor="right",
            y=button_layer_1_height,
            yanchor="top"
        ),
        dict(
            buttons=list([
                dict(
                    args=["Status", "2"],
                    label="アラーム発生／非常停止ボタンが押された時 	00:00:04",
                    method="restyle"
                )
            ]),
            type = "buttons",
            direction="right",
            pad={"r": 50, "t": 50},
            showactive=True,
            x=0.9,
            xanchor="right",
            y=button_layer_2_height,
            yanchor="top"
        ),
        dict(
            buttons=list([
                dict(
                    args=["Status", "3"],
                    label="ー 	00:00:01",
                    method="restyle"
                )
            ]),type = "buttons",
            direction="right",
            pad={"r": 50, "t": 50},
            showactive=True,
            x=0.9,
            xanchor="right",
            y=button_layer_3_height,
            yanchor="top"),
        dict(
            buttons=list([
                dict(
                    args=["Status", "4"],
                    label="加工中 	08:13:35",
                    method="restyle"
                )
            ]),type = "buttons",
            direction="right",
            pad={"r": 50, "t": 50},
            showactive=True,
            x=0.9,
            xanchor="right",
            y=button_layer_4_height,
            yanchor="top"),
        dict(
            buttons=list([
                dict(
                    args=["Status", "5"],
                    label="加工終了 	03:26:59",
                    method="restyle"
                )
            ]),type = "buttons",
            direction="right",
            pad={"r": 50, "t": 50},
            showactive=True,
            x=0.9,
            xanchor="right",
            y=button_layer_5_height,
            yanchor="top"),
        dict(
            buttons=list([
                dict(
                    args=["Status", "6"],
                    label="電源は入っているが非稼働／非稼働 	02:51:36",
                method="restyle"
                )
            ]),type = "buttons",
            direction="right",
            pad={"r": 50, "t": 50},
            showactive=True,
            x=0.9,
            xanchor="right",
            y=button_layer_6_height,
            yanchor="top")
    ]
)

fig2.update_layout(
    annotations=[
        dict(text="Status", x=0, xref="paper", y=1.1, yref="paper",
                             align="right", showarrow=False)
    ])

fig2.data[0].domain = {'x': [0, 0.5], 'y': [0.5, 0.5]}
fig2.show()


In [ ]:
with open('Sensors.html','a') as f:
    f.write(fig2.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(fig_sub.to_html(full_html=False, include_plotlyjs='cdn'))